# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
!pip install opencv-python numpy tensorflow scikit-learn matplotlib pillow pandas mediapipe ffmpeg-python

In [3]:
import cv2
import os
import time
import mediapipe as mp
import csv
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


Data Collection

In [4]:


# -------------------------
# Settings
# -------------------------
CLASSES = ["Yes", "No"]   # <-- Add more categories here
SAVE_DIR = "dataset"
VIDEO_COUNT = 10
DURATION = 3
PAUSE_DURATION = 3
FPS = 30
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720
# -------------------------

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
cap.set(cv2.CAP_PROP_FPS, FPS)

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

print("Starting multi-category recording...")

for CLASS_NAME in CLASSES:

    print(f"\n===== START CATEGORY: {CLASS_NAME} =====")

    # Create folders
    class_folder = os.path.join(SAVE_DIR, CLASS_NAME)
    os.makedirs(class_folder, exist_ok=True)
    keypoints_folder = os.path.join(class_folder, "keypoints")
    os.makedirs(keypoints_folder, exist_ok=True)

    for i in range(1, VIDEO_COUNT + 1):

        # CSV file for keypoints
        csv_filename = f"{CLASS_NAME}_{i}_keypoints.csv"
        csv_filepath = os.path.join(keypoints_folder, csv_filename)

        csv_file = open(csv_filepath, "w", newline="")
        csv_writer = csv.writer(csv_file)

        # Header
        header = []
        for hand in ['left', 'right']:
            for j in range(21):
                header += [f"{hand}_x{j}", f"{hand}_y{j}", f"{hand}_z{j}"]
        header.append("label")
        csv_writer.writerow(header)

        # Video Writer
        video_filename = f"{CLASS_NAME}_{i}.mp4"
        video_path = os.path.join(class_folder, video_filename)
        fourcc = cv4 = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(video_path, fourcc, FPS, (FRAME_WIDTH, FRAME_HEIGHT))

        print(f"\nRecording {CLASS_NAME} - Video {i}/{VIDEO_COUNT}")

        start_time = time.time()

        while time.time() - start_time < DURATION:
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.flip(frame, 1)

            # Process with MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(rgb_frame)

            left_hand_kp = [(0, 0, 0)] * 21
            right_hand_kp = [(0, 0, 0)] * 21

            if result.multi_hand_landmarks and result.multi_handedness:
                for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    label = handedness.classification[0].label
                    hand_kp = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]

                    if label == "Left":
                        left_hand_kp = hand_kp
                    else:
                        right_hand_kp = hand_kp

                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Save keypoints
            row = [coord for kp in left_hand_kp + right_hand_kp for coord in kp]
            row.append(CLASS_NAME)
            csv_writer.writerow(row)

            # Overlay info
            sec_left = int(DURATION - (time.time() - start_time) + 1)
            cv2.putText(frame, f"Category: {CLASS_NAME}", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(frame, f"Recording Video {i}/{VIDEO_COUNT}", (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Time Left: {sec_left}s", (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Show & write
            cv2.imshow("Recorder", frame)
            out.write(frame)

            if cv2.waitKey(1) == ord('q'):
                print("Stopped by user.")
                cap.release()
                cv2.destroyAllWindows()
                raise SystemExit()

        out.release()
        csv_file.close()

        print(f"Saved video: {video_path}")
        print(f"Saved keypoints: {csv_filepath}")

        # Pause between videos
        print(f"Waiting {PAUSE_DURATION} seconds...")
        pause_start = time.time()
        while time.time() - pause_start < PAUSE_DURATION:
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.flip(frame, 1)
            remaining = int(PAUSE_DURATION - (time.time() - pause_start))
            cv2.putText(frame, f"Next video in {remaining}s", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.imshow("Recorder", frame)
            if cv2.waitKey(1) == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                raise SystemExit()

cap.release()
cv2.destroyAllWindows()
print("\nAll categories recorded successfully!")


Starting multi-category recording...

===== START CATEGORY: Yes =====

Recording Yes - Video 1/10
Saved video: dataset\Yes\Yes_1.mp4
Saved keypoints: dataset\Yes\keypoints\Yes_1_keypoints.csv
Waiting 3 seconds...

Recording Yes - Video 2/10
Saved video: dataset\Yes\Yes_2.mp4
Saved keypoints: dataset\Yes\keypoints\Yes_2_keypoints.csv
Waiting 3 seconds...

Recording Yes - Video 3/10
Saved video: dataset\Yes\Yes_3.mp4
Saved keypoints: dataset\Yes\keypoints\Yes_3_keypoints.csv
Waiting 3 seconds...

Recording Yes - Video 4/10
Saved video: dataset\Yes\Yes_4.mp4
Saved keypoints: dataset\Yes\keypoints\Yes_4_keypoints.csv
Waiting 3 seconds...


SystemExit: 

C:\Users\kongm\PycharmProjects\SignLanguage\.venv3\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Data Preprocessing

In [5]:
# -------------------------
# Settings
# -------------------------
DATASET_DIR = "dataset"    # folder with category subfolders
SEQUENCE_LENGTH = 30       # number of frames per sample
# -------------------------

def fix_sequence_length(sequence, target_len):
    """Pad or truncate keypoint sequence to a fixed length."""
    if len(sequence) > target_len:
        return sequence[:target_len]
    elif len(sequence) < target_len:
        pad = np.zeros((target_len - len(sequence), sequence.shape[1]))
        return np.vstack([sequence, pad])
    return sequence

X = []
y = []

# Loop through categories
for category in os.listdir(DATASET_DIR):
    kp_dir = os.path.join(DATASET_DIR, category, "keypoints")
    if not os.path.isdir(kp_dir):
        continue

    for csv_file in os.listdir(kp_dir):
        csv_path = os.path.join(kp_dir, csv_file)
        df = pd.read_csv(csv_path)

        # Remove label column
        keypoints = df.iloc[:, :-1].values.astype(np.float32)

        # Fix sequence length
        keypoints = fix_sequence_length(keypoints, SEQUENCE_LENGTH)

        X.append(keypoints)
        y.append(category)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)  # (samples, SEQUENCE_LENGTH, 126)
print("y shape:", y.shape)  # (samples,)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

print("Classes:", label_encoder.classes_)
print("y_onehot shape:", y_onehot.shape)

# Save preprocessed data
np.save("X_keypoints.npy", X)
np.save("y_labels.npy", y_onehot)
print("Preprocessed data saved to X_keypoints.npy and y_labels.npy")

X shape: (25, 30, 126)
y shape: (25,)
Classes: ['Hello' 'No' 'Yes']
y_onehot shape: (25, 3)
Preprocessed data saved to X_keypoints.npy and y_labels.npy


Modeling

In [6]:
X = np.load("X_keypoints.npy")      # shape: (samples, SEQUENCE_LENGTH, 126)
y = np.load("y_labels.npy")         # shape: (samples, num_classes)

print("X shape:", X.shape)
print("y shape:", y.shape)

# -------------------------
# Model definition
# -------------------------
SEQUENCE_LENGTH = X.shape[1]
FEATURES = X.shape[2]
NUM_CLASSES = y.shape[1]

model = Sequential()

# 1D Convolutions over keypoints per frame
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(SEQUENCE_LENGTH, FEATURES)))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# LSTM for temporal information
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------
# Callbacks
# -------------------------
checkpoint = ModelCheckpoint("sign_language_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# -------------------------
# Train-test split
# -------------------------
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------
# Train the model
# -------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[checkpoint, early_stop]
)

X shape: (25, 30, 126)
y shape: (25, 3)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            24256     
                                                                 
 conv1d_1 (Conv1D)           (None, 26, 128)           24704     
                                                                 
 max_pooling1d (MaxPooling1  (None, 13, 128)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 128)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
             

C:\Users\kongm\PycharmProjects\SignLanguage\.venv3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 51ms/step - loss: 0.9819 - accuracy: 0.5000 - val_loss: 0.9407 - val_accuracy: 0.8000
Epoch 5/50
1/2 [==============>...............] - ETA: 0s - loss: 0.9353 - accuracy: 0.5625
Epoch 5: val_accuracy did not improve from 0.80000
2/2 [==============================] - 0s 34ms/step - loss: 0.9218 - accuracy: 0.5500 - val_loss: 0.9292 - val_accuracy: 0.6000
Epoch 6/50
1/2 [==============>...............] - ETA: 0s - loss: 0.9133 - accuracy: 0.5625
Epoch 6: val_accuracy did not improve from 0.80000
2/2 [==============================] - 0s 34ms/step - loss: 0.9308 - accuracy: 0.6000 - val_loss: 0.9256 - val_accuracy: 0.6000
Epoch 7/50
1/2 [==============>...............] - ETA: 0s - loss: 0.9151 - accuracy: 0.5625
Epoch 7: val_accuracy did not improve from 0.80000
2/2 [==============================] - 0s 31ms/step - loss: 0.9175 - accuracy: 0.6000 - val_loss: 0.9088 - val_accuracy: 0.6000
Epoch 8/50
1/2 [==============>...............] - ETA: 0s -

In [7]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import os

# -------------------------
# Settings
# -------------------------
MODEL_PATH = "sign_language_model.h5"   # your trained model
DATASET_DIR = "dataset"                 # folder used in preprocessing
SEQUENCE_LENGTH = 30
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720
FPS = 30
# -------------------------

# Load trained model
model = load_model(MODEL_PATH)

# Dynamically get class names from dataset folder
CLASSES = [d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))]
CLASSES.sort()  # ensure consistent order

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
cap.set(cv2.CAP_PROP_FPS, FPS)

# Sequence buffer
sequence = []

print("Starting real-time gesture test. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue
    frame = cv2.flip(frame, 1)

    # MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    # Extract keypoints
    left_hand_kp = [(0, 0, 0)] * 21
    right_hand_kp = [(0, 0, 0)] * 21

    if result.multi_hand_landmarks and result.multi_handedness:
        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            label = handedness.classification[0].label
            hand_kp = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            if label == "Left":
                left_hand_kp = hand_kp
            else:
                right_hand_kp = hand_kp
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Flatten keypoints and add to sequence
    keypoints = [coord for kp in left_hand_kp + right_hand_kp for coord in kp]
    sequence.append(keypoints)

    # Keep last SEQUENCE_LENGTH frames
    if len(sequence) > SEQUENCE_LENGTH:
        sequence = sequence[-SEQUENCE_LENGTH:]

    # Make prediction when sequence is full
    if len(sequence) == SEQUENCE_LENGTH:
        input_data = np.expand_dims(sequence, axis=0)  # shape: (1, SEQUENCE_LENGTH, 126)
        prediction = model.predict(input_data, verbose=0)
        class_id = np.argmax(prediction)
        class_name = CLASSES[class_id]
        confidence = prediction[0][class_id]
        cv2.putText(frame, f"{class_name} ({confidence*100:.1f}%)", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    cv2.imshow("Real-Time Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
hands.close()


Starting real-time gesture test. Press 'q' to quit.
